In [138]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

class PropellerDataset(Dataset):
    def __init__(self, root='data'):
        self.points = np.load(os.path.join(root, 'points.npy')).astype(np.float32)
        self.labels = np.load(os.path.join(root, 'labels.npy')).astype(np.int64)

    def __len__(self):
        return 1

    def __getitem__(self, idx):
        return self.points, self.labels







In [140]:
def knn(x, k):
    inner = -2 * torch.matmul(x.transpose(2, 1), x)
    xx = torch.sum(x ** 2, dim=1, keepdim=True)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)
    idx = pairwise_distance.topk(k=k, dim=-1)[1]
    return idx

def get_graph_feature(x, k=20, idx=None):
    batch_size, num_dims, num_points = x.size()
    k = min(k, num_points - 1)
    if idx is None:
        idx = knn(x, k=k)
    device = x.device
    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1) * num_points
    idx = idx + idx_base
    idx = idx.view(-1)
    x = x.transpose(2, 1).contiguous()
    feature = x.view(batch_size * num_points, -1)[idx, :]
    feature = feature.view(batch_size, num_points, k, num_dims)
    x = x.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1)
    feature = torch.cat((feature - x, x), dim=3).permute(0, 3, 1, 2).contiguous()
    return feature




In [142]:
class DGCNN(nn.Module):
    def __init__(self, args, output_channels=4):
        super(DGCNN, self).__init__()
        self.k = args.k
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.bn5 = nn.BatchNorm1d(args.emb_dims)

        self.conv1 = nn.Sequential(nn.Conv2d(6, 64, 1, bias=False), self.bn1, nn.LeakyReLU(0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(128, 64, 1, bias=False), self.bn2, nn.LeakyReLU(0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(128, 128, 1, bias=False), self.bn3, nn.LeakyReLU(0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(256, 256, 1, bias=False), self.bn4, nn.LeakyReLU(0.2))
        self.conv5 = nn.Sequential(nn.Conv1d(512, args.emb_dims, 1, bias=False), self.bn5, nn.LeakyReLU(0.2))
        self.dp1 = nn.Dropout(p=args.dropout)
        self.conv_seg = nn.Conv1d(args.emb_dims, output_channels, 1)

    def forward(self, x):
        batch_size, _, num_points = x.size()
        x1 = self.conv1(get_graph_feature(x, k=self.k)).max(dim=-1)[0]
        x2 = self.conv2(get_graph_feature(x1, k=self.k)).max(dim=-1)[0]
        x3 = self.conv3(get_graph_feature(x2, k=self.k)).max(dim=-1)[0]
        x4 = self.conv4(get_graph_feature(x3, k=self.k)).max(dim=-1)[0]
        x_cat = torch.cat((x1, x2, x3, x4), dim=1)
        x = self.conv5(x_cat)
        x = self.dp1(x)
        x = self.conv_seg(x)
        return x.transpose(2, 1).contiguous()



In [144]:
class Args:
    def __init__(self):
        self.k = 20
        self.emb_dims = 1024
        self.dropout = 0.5

args = Args()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DGCNN(args, output_channels=4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

dataset = PropellerDataset()
loader = DataLoader(dataset, batch_size=1, shuffle=False)



In [150]:
from collections import defaultdict

for epoch in range(20):
    model.train()
    total_loss = 0.0
    correct = total = 0

    # Initialize per-class stats
    correct_per_class = defaultdict(int)
    total_per_class = defaultdict(int)

    for points, labels in loader:
        points = points.to(device)        # [1, N, 3]
        labels = labels.to(device)        # [1, N]
        points = points.permute(0, 2, 1)  # [1, 3, N]

        outputs = model(points)           # [1, N, 4]
        loss = criterion(outputs.view(-1, 4), labels.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        preds = outputs.argmax(dim=2)  # [1, N]
        correct += (preds == labels).sum().item()
        total += labels.numel()

        # Count per-class stats
        for p, l in zip(preds.view(-1), labels.view(-1)):
            total_per_class[l.item()] += 1
            if p.item() == l.item():
                correct_per_class[l.item()] += 1

    acc = 100 * correct / total
    print(f"\n📉 Epoch {epoch+1}, Loss: {total_loss:.4f}, ✅ Accuracy: {acc:.2f}%")

    # Per-class accuracy
    for cls in sorted(total_per_class.keys()):
        class_acc = 100 * correct_per_class[cls] / total_per_class[cls]
        print(f"  🔹 Class {cls}: {class_acc:.2f}% accuracy ({correct_per_class[cls]}/{total_per_class[cls]})")





📉 Epoch 1, Loss: 0.0050, ✅ Accuracy: 99.85%
  🔹 Class 0: 99.14% accuracy (346/349)
  🔹 Class 1: 100.00% accuracy (837/837)
  🔹 Class 3: 100.00% accuracy (856/856)

📉 Epoch 2, Loss: 0.0056, ✅ Accuracy: 99.80%
  🔹 Class 0: 99.43% accuracy (347/349)
  🔹 Class 1: 100.00% accuracy (837/837)
  🔹 Class 3: 99.77% accuracy (854/856)

📉 Epoch 3, Loss: 0.0044, ✅ Accuracy: 99.90%
  🔹 Class 0: 99.43% accuracy (347/349)
  🔹 Class 1: 100.00% accuracy (837/837)
  🔹 Class 3: 100.00% accuracy (856/856)

📉 Epoch 4, Loss: 0.0038, ✅ Accuracy: 99.90%
  🔹 Class 0: 99.43% accuracy (347/349)
  🔹 Class 1: 100.00% accuracy (837/837)
  🔹 Class 3: 100.00% accuracy (856/856)

📉 Epoch 5, Loss: 0.0037, ✅ Accuracy: 99.90%
  🔹 Class 0: 99.43% accuracy (347/349)
  🔹 Class 1: 100.00% accuracy (837/837)
  🔹 Class 3: 100.00% accuracy (856/856)

📉 Epoch 6, Loss: 0.0029, ✅ Accuracy: 100.00%
  🔹 Class 0: 100.00% accuracy (349/349)
  🔹 Class 1: 100.00% accuracy (837/837)
  🔹 Class 3: 100.00% accuracy (856/856)

📉 Epoch 7, Los

In [152]:
!pip install open3d
import open3d as o3d
import torch
import numpy as np

def mesh_to_pointcloud(filepath, num_points=2048):
    mesh = o3d.io.read_triangle_mesh(filepath)
    mesh.compute_vertex_normals()

    # Sample points uniformly
    pcd = mesh.sample_points_uniformly(number_of_points=num_points)
    points = np.asarray(pcd.points, dtype=np.float32)
    
    # [1, 3, N] format for DGCNN
    points = torch.tensor(points).unsqueeze(0).permute(0, 2, 1)
    return points, pcd  # return pcd if you want to visualize
    


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [160]:
model.eval()
points_a, _ = mesh_to_pointcloud("10x4M-LH-PERF.STL")
points_b, _ = mesh_to_pointcloud("11x6M-LH-PERF (1).STL")
points_c, _ = mesh_to_pointcloud("14x13M.STL")


In [164]:
with torch.no_grad():
    preds_a = model(points_a).argmax(dim=2).view(-1)
    preds_b = model(points_b).argmax(dim=2).view(-1)
    preds_c = model(points_c).argmax(dim=2).view(-1)

# Compare predictions
def compute_error(pred1, pred2):
    min_len = min(len(pred1), len(pred2))
    return 100.0 * (pred1[:min_len] != pred2[:min_len]).sum().item() / min_len

print(f"🔹 Error A vs B: {compute_error(preds_a, preds_b):.2f}%")
print(f"🔹 Error A vs C: {compute_error(preds_a, preds_c):.2f}%")

🔹 Error A vs B: 88.28%
🔹 Error A vs C: 88.28%
